In [1]:
import gspread
import os
import dotenv
import requests, json
import pandas as pd
import unicodedata

from dotenv import load_dotenv

In [32]:
with open('data.json', 'rt', encoding='UTF8') as json_file:
    json_data = json.load(json_file)
json_data

{'attandance_week': 8,
 'attandance_notion_database_id': '3a2a9742f1fa40f18c432a06401310cf',
 'class': [{'class_name': '데이터 분석 개념반',
   'day': '수',
   'time': 20,
   'notion_database_id': '2de38e6d6c4d4da19cead9225d67d992',
   'week': 8},
  {'class_name': 'Python 문법 기초반',
   'day': '수',
   'time': 21,
   'notion_database_id': '72873837961d49ffa78f6e61eb94988f',
   'week': 8},
  {'class_name': 'SQL 문법 기초반',
   'day': '수',
   'time': 22,
   'notion_database_id': 'e4bbcb936d5043449a691b754c5f9e5e',
   'week': 8},
  {'class_name': '데이터 분석 입문반(머신러닝)',
   'day': '토',
   'time': 11,
   'notion_database_id': '46a7fd9ee120440b9bdca78ab9e35a9d',
   'week': 8},
  {'class_name': '데이터 분석 입문반(통계)',
   'day': '토',
   'time': 12,
   'notion_database_id': 'ff25b42f13fd4b28978b5cfe05cc34cd',
   'week': 8},
  {'class_name': '데이터 분석 모델링반',
   'day': '일',
   'time': 18,
   'notion_database_id': 'd763f50495164b96b9e8d4be5c640dad',
   'week': 8},
  {'class_name': '데이터 분석 전처리반(판다스)',
   'day': '일',
   'time':

In [25]:
df = pd.read_csv('zoom_logs/2주차/Python 문법 기초반.csv')
df['참가 시간'] = pd.to_datetime(df['참가 시간'])
df['나간 시간'] = pd.to_datetime(df['나간 시간'])
df

C:\Users\user\AppData\Local\Temp\ipykernel_28984\3829361690.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['참가 시간'] = pd.to_datetime(df['참가 시간'])
C:\Users\user\AppData\Local\Temp\ipykernel_28984\3829361690.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['나간 시간'] = pd.to_datetime(df['나간 시간'])


,이름(원래 이름),사용자 이메일,참가 시간,나간 시간,기간(분),게스트,대기실
0,jun datachef,official.datachef@gmail.com,2024-03-06 20:55:31,2024-03-06 21:55:10,60,아니요,아니요
1,안경민_55,NaN,2024-03-06 20:56:12,2024-03-06 21:55:09,59,예,아니요
2,Zoom 사용자,NaN,2024-03-06 20:56:13,2024-03-06 21:55:10,59,예,아니요
3,하태훈 97,NaN,2024-03-06 20:56:20,2024-03-06 21:55:10,59,예,아니요
4,김근영_58,NaN,2024-03-06 20:56:20,2024-03-06 21:55:10,59,예,아니요
...,...,...,...,...,...,...,...
220,김예지_12,NaN,2024-03-06 21:29:17,2024-03-06 21:30:28,2,예,아니요
221,김예지_12,NaN,2024-03-06 21:30:31,2024-03-06 21:55:09,25,예,아니요
222,조수아,NaN,2024-03-06 21:30:50,2024-03-06 21:55:10,25,예,아니요
223,강지웅,NaN,2024-03-06 21:34:45,2024-03-06 21:55:10,21,예,아니요


In [34]:
df['나간 시간'][0].hour - 8

13

In [26]:
df.dtypes

이름(원래 이름)            object
사용자 이메일              object
참가 시간        datetime64[ns]
나간 시간        datetime64[ns]
기간(분)                 int64
게스트                  object
대기실                  object
dtype: object

In [20]:
df['참가 시간'][0] - df['나간 시간'][0]

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [35]:
load_dotenv()
notion_token = os.environ.get('NOTION_API')
notion_header = {
    "Authorization":"Bearer "+ notion_token,
    "Notion-Version":"2022-06-28"
}
notion_database_read_url = f"https://api.notion.com/v1/databases/989be6df0f7e401cb07c186e0c6e2306/query"

In [36]:
params = {}
pages_and_databases = []

while True:
    search_response = requests.post(notion_database_read_url, json=params, headers=notion_header)
    if search_response.ok is False: break
    search_response_obj = search_response.json()
    pages_and_databases.extend(search_response_obj.get("results"))
    params["start_cursor"] = search_response_obj.get("next_cursor")

In [37]:
column_name = [x for x in list(pages_and_databases[0]['properties'].keys()) if f"({6}주차)" in x][0]
week_hw_data = []
for info in pages_and_databases:
    name = ''
    if info['properties']['이름']['title']:
        name = unicodedata.normalize('NFC', info['properties']['이름']['title'][0]['plain_text'])
    week_hw_data.append(name)

In [45]:
json_file_path = os.environ.get('GCP_JSON_PATH')

gc = gspread.service_account(json_file_path)
spreadsheet_url = os.environ.get('SHEET_URL')
doc = gc.open_by_url(spreadsheet_url)



In [48]:
attendance = doc.worksheet("데이터 분석 입문반(통계)")
range_list = [x.value for x in attendance.range('A4:A300') if x.value != '']
len(range_list)

107

In [47]:
attendance = doc.worksheet("데이터 분석 전처리 적용반")
range_list = [x.value for x in attendance.range('A4:A300') if x.value != '']
len(range_list)

NameError: name 'doc' is not defined

In [ ]:
import time
def cellDection(spread_sheet_type : str, week : int)->list:
    #수식 6+4*week+weight
    #예외 AA
    #(6+4*week+weight)/(ord('Z')-ord('A'))의 몫과 나머지 사용해서 chr
    #chr((ord('A')-1) + (5+4*(13-1))//(ord('Z')+1-ord('A'))), chr(ord('A')+(5+4*(13-1))%(ord('Z')+1-ord('A')))
    alphabet_count = ord('Z')+1-ord('A')
    if spread_sheet_type == "출석": bais = 0
    elif spread_sheet_type == "필수": bais = 2
    elif spread_sheet_type == "복습": bais = 3
    else: return None
    cell_cal = 5+4*(week-1)+bais

    if cell_cal//alphabet_count == 0:
        return [chr(ord('A')+cell_cal%alphabet_count), 4]
    else:
        return [f"{chr((ord('A')-1)+cell_cal//alphabet_count)}{chr((ord('A'))+cell_cal%alphabet_count)}",4]


def accuracyCalculator(type_name : str, train_list : list, target_list : list)->float:
    current_count = 0

    train_parsing = [x.value for x in train_list]
    target_parsing = [x.value for x in target_list]

    for x in range(len(train_parsing)):
        if type_name == '출석':
            if train_parsing[x] == 'X' and target_parsing[x] == '사유작성': current_count += 1
            elif train_parsing[x] == target_parsing[x]: current_count += 1
        else:
            if train_parsing[x] == 'X' and target_parsing[x] == '지각': current_count += 1
            elif train_parsing[x] == target_parsing[x]: current_count += 1
    
    return current_count/len(target_list)



json_file_path = os.environ.get('GCP_JSON_PATH')

gc = gspread.service_account(json_file_path)
train_spreadsheet_url = "https://docs.google.com/spreadsheets/d/1JzXQ2LySIkdA6Zgp0Xsobzw9qvlYQFeNWv0fOsneTTs/edit#gid=1868239497"
target_spreadsheet_url = "https://docs.google.com/spreadsheets/d/1W4qZDZcWHq4AzTjETQX6vN1yo2HFcN_Ko_bSmd0WnxY/edit#gid=254208571"
train_doc = gc.open_by_url(train_spreadsheet_url)
target_doc = gc.open_by_url(target_spreadsheet_url)

accuracy_list = []

for class_name in ["Python 문법 기초반", "SQL 문법 기초반", "데이터 분석 입문반(머신러닝)", "데이터 분석 입문반(통계)", "데이터 분석 전처리반(판다스)", "데이터 분석 전처리 적용반", "데이터 분석 모델링반", "데이터 분석 개념반"]:
    train_class_sheet = train_doc.worksheet(class_name)
    target_class_sheet = target_doc.worksheet(class_name)

    class_len = len([x.value for x in target_class_sheet.range('A4:A400') if x.value != ''])-1  

    for week in range(3,7):
        for type_name in ["출석", "필수", "복습"]:
            cell_location = cellDection(type_name, week)
            train_data = train_class_sheet.range(f'{cell_location[0]}{cell_location[1]}:{cell_location[0]}{cell_location[1]+class_len}')
            target_data = target_class_sheet.range(f'{cell_location[0]}{cell_location[1]}:{cell_location[0]}{cell_location[1]+class_len}')
            accuracy_list.append([class_name, week, type_name, class_len, accuracyCalculator(type_name, train_data, target_data)])
            time.sleep(10)




In [ ]:
accuracy_list

[['Python 문법 기초반', 3, '출석', 224, 0.8444444444444444],
 ['Python 문법 기초반', 3, '필수', 224, 0.9466666666666667],
 ['Python 문법 기초반', 3, '복습', 224, 0.9422222222222222],
 ['Python 문법 기초반', 4, '출석', 224, 0.8888888888888888],
 ['Python 문법 기초반', 4, '필수', 224, 0.9822222222222222],
 ['Python 문법 기초반', 4, '복습', 224, 0.9866666666666667],
 ['Python 문법 기초반', 5, '출석', 224, 0.8488888888888889],
 ['Python 문법 기초반', 5, '필수', 224, 0.0],
 ['Python 문법 기초반', 5, '복습', 224, 0.9644444444444444],
 ['Python 문법 기초반', 6, '출석', 224, 0.6933333333333334],
 ['Python 문법 기초반', 6, '필수', 224, 0.0],
 ['Python 문법 기초반', 6, '복습', 224, 0.9555555555555556],
 ['SQL 문법 기초반', 3, '출석', 145, 0.8356164383561644],
 ['SQL 문법 기초반', 3, '필수', 145, 0.0],
 ['SQL 문법 기초반', 3, '복습', 145, 0.9657534246575342],
 ['SQL 문법 기초반', 4, '출석', 145, 0.8561643835616438],
 ['SQL 문법 기초반', 4, '필수', 145, 0.9794520547945206],
 ['SQL 문법 기초반', 4, '복습', 145, 0.9794520547945206],
 ['SQL 문법 기초반', 5, '출석', 145, 0.8835616438356164],
 ['SQL 문법 기초반', 5, '필수', 145, 0.0],
 ['S

In [49]:
pd.DataFrame(accuracy_list).to_csv('accuracy_data.csv')

In [50]:
pd.DataFrame(accuracy_list).tail(20)

,0,1,2,3,4
76,데이터 분석 모델링반,4,필수,138,0.956835
77,데이터 분석 모델링반,4,복습,138,0.949640
78,데이터 분석 모델링반,5,출석,138,0.949640
79,데이터 분석 모델링반,5,필수,138,0.000000
80,데이터 분석 모델링반,5,복습,138,0.949640
81,데이터 분석 모델링반,6,출석,138,0.935252
82,데이터 분석 모델링반,6,필수,138,0.000000
83,데이터 분석 모델링반,6,복습,138,0.000000
84,데이터 분석 개념반,3,출석,49,0.900000
85,데이터 분석 개념반,3,필수,49,0.000000


In [ ]:
# 출석 정확도가 낮은 경우 -> 준님이 대기실에 오래있었음 ㅋㅋ;;;;;

In [20]:
for i, val in enumerate(attendance):
    print(i, val)

TypeError: 'Worksheet' object is not iterable

In [ ]:
week_hw_data

In [19]:
type(attendance)

gspread.worksheet.Worksheet